In [2]:
#import libraries
import pickle
from collections import defaultdict
import numpy as np
import pandas as pd
import os
import h5py
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
# import torch
# import torchtext
# import torch.nn as nn
# import torch.nn.functional as F
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

[nltk_data] Downloading package punkt to /home/eileen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
os.listdir('data')

['AMAZON_FASHION.json.gz', 'data.zip', 'mosei', 'aclImdb_v1.tar.gz']

In [5]:
#load data

#file located here: https://drive.google.com/file/d/1tcVYIMcZdlDzGuJvnMtbMchKIK9ulW1P/view

#make data directory if doesn't exist in path folder
os.makedirs('data', exist_ok=True)

#download meld and mosei zip files for data (mosei is from CMU, meld is friends episodes)
if('mosei' in  os.listdir('data')):
    pass
else:
    !file=1tcVYIMcZdlDzGuJvnMtbMchKIK9ulW1P && wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='${file} -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="${file} -O data/data.zip && rm -rf /tmp/cookies.txt

    #unzip data
    !unzip data/data.zip -d data/mosei


#train
text_tr = pickle.load(open('data/mosei/MOSEI/train_sentences.p',"rb"))
emotion_tr = pickle.load(open('data/mosei/MOSEI/train_emotion.p', "rb"))
sentiment_tr = pickle.load(open('data/mosei/MOSEI/train_sentiment.p', "rb"))
#change sentimement to be binary
sentiment_tr = {k: np.sign(v) for k,v in sentiment_tr.items()}


#valid
text_val = pickle.load(open('data/mosei/MOSEI/valid_sentences.p',"rb"))
emotion_val = pickle.load(open('data/mosei/MOSEI/valid_emotion.p', "rb"))
sentiment_val = pickle.load(open('data/mosei/MOSEI/valid_sentiment.p', "rb"))
#change sentiement to be binary
sentiment_val = {k: np.sign(v) for k,v in sentiment_val.items()}

In [6]:
col_name = ['text','emotion','sentiment']
#create pandas dataframe
df_train = pd.DataFrame.from_dict([text_tr, emotion_tr, sentiment_tr]).T
df_train.columns = col_name
df_train.head

df_val= pd.DataFrame.from_dict([text_val, emotion_val, sentiment_val]).T
df_val.columns = col_name
df_val.head

<bound method NDFrame.head of                                                               text  \
--qXJuDtHPw[0]   [writer, sp, i, see, that, a, writer, is, some...   
-571d8cVauQ[0]   [sp, this, is, rhett, reiger, white, caspian, ...   
-571d8cVauQ[1]   [create, sp, however, again, on, a, low, budge...   
-571d8cVauQ[2]   [was, sp, so, very, important, to, sp, go, out...   
-I_e4mIh0yE[0]   [entrepreneurship, sp, in, the, united, states...   
...                                                            ...   
zjYEBwXGD8I[15]  [sp, with, sp, unlike, some, other, companies,...   
zjYEBwXGD8I[16]  [their, fees, you, can, see, all, of, them, on...   
zjYEBwXGD8I[17]  [website, sp, they, also, offer, sp, a, money,...   
zk2jTlAtvSU[0]   [sp, they, differ, from, the, articles, of, in...   
zk2jTlAtvSU[1]   [sp, they, also, discuss, sp, how, many, direc...   

                                                      emotion sentiment  
--qXJuDtHPw[0]           [0.6666667, 0.0, 0.0, 0.0, 0.0

In [8]:
#explore training data
y1_train = df_train['emotion']
y2_train = df_train['sentiment']
x_train = df_train['text']

print("Distribution of emotions:", y1_train.sum()/len(y1_train))
print("Distribution of Sentiment", y2_train.value_counts())

Distribution of emotions: [0.48713377 0.1608987  0.15779537 0.04880444 0.1243045  0.04050551]
Distribution of Sentiment [1.0]     8050
[-1.0]    4739
[0.0]     3538
Name: sentiment, dtype: int64


In [12]:
#convert vector series into dataframe
y1_df = pd.Series(y1_train).apply(lambda x: pd.Series(x))
y1_df.columns = ['emo1', 'emo2', 'emo3', 'emo4', 'emo5', 'emo6']

#detect presense of emotion
y1_presence = y1_df.where(y1_df==0, 1)

print("Distribution of emotion presence:", y1_presence.sum()/len(y1_presence))

Distribution of emotion presence: emo1    0.535003
emo2    0.261469
emo3    0.215961
emo4    0.100570
emo5    0.180989
emo6    0.081521
dtype: float32


In [31]:
#convert y values to numpy arrays
y1_bin_train = y1_presence.to_numpy()
y1_val_train = y1_train.to_numpy()

y2_train = y2_train.to_numpy()